In [184]:
import numpy as np
import csv
import os
import os.path as path
from tqdm import tqdm, trange
import cv2
import time

In [185]:
pred_ext = '_all.csv'
path_pred = '/Users/lucas/Desktop/test_mAP_2'
path_image = '/Users/lucas/Desktop/test_mAP_2'

path_nms = '/Users/lucas/Desktop/yolo_test_pred'

classes = ["Person_sitting", "Cyclist", "Pedestrian", "Van", "Truck", "Misc", "DontCare", "Car", "Tram"]

display_result = False

# Standard NMS
std_obj_threshold = 0.3
std_iou_threshold = 0.3

# Soft NMS (Linear)
soft_lin_obj_threshold = 0.3
soft_lin_iou_threshold = 0.3

# Soft NMS (Gaussian)
soft_gaus_obj_threshold = 0.3
soft_gaus_iou_threshold = 0.3
soft_gaus_sigma = 0.4

# Seq NMS

# Soft Seq NMS

In [87]:
def import_pred(file_path):
    with open(file_path) as csvfile:
        csvreader = csv.reader(csvfile, delimiter=',')
        list_row = [row for row in csvreader]
        row_count = len(list_row)
        col_count = len(list_row[0])

        bboxes = np.zeros((row_count, col_count))

        for idx, row in enumerate(list_row):
            bboxes[idx,:] = np.asarray([float(num) for num in row])

    return bboxes

In [123]:
def convert_array_to_pred(bboxes, im_shape, classes):
    pred_list = []
    num_boxes = bboxes.shape[0]
    
    for idx in range(num_boxes):
        if np.sum(bboxes[idx, 5:]) > 0:
            x, y, w, h = bboxes[idx, :4]
            
            xmin  = int((x - w/2) * im_shape[1])
            xmax  = int((x + w/2) * im_shape[1])
            ymin  = int((y - h/2) * im_shape[0])
            ymax  = int((y + h/2) * im_shape[0])
            label = classes[np.argmax(bboxes[idx, 5:])].encode("utf-8")
            score = np.max(bboxes[idx, 5:])
            pred_list.append([label, xmin, ymin, xmax, ymax, score])
    
    return pred_list

In [182]:
def nms_fast(boxes, iou_threshold, obj_threshold, sigma=0.5, method=''):
    '''
    Inspired from: https://www.pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/ Malisiewicz et al.
    boxes = [x, y, w, h, classe_score] all need to be float
    '''
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
 
    # grab the coordinates of the bounding boxes and the score for the class
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,0] + boxes[:,2]  # because boxes[:,2] is w
    y2 = boxes[:,1] + boxes[:,3]  # because boxes[:,3] is h
    sc = boxes[:,4]
    
    sc_output = np.zeros(boxes.shape[0])
    
    if np.max(sc) < obj_threshold: # Test if some boxes need to be considered
        return [], [], sc_output
    else:
        # initialize the list of picked indexes
        pick = []
        
        # compute the area of the bounding boxes and sort the bounding
        # boxes by the bottom-right y-coordinate of the bounding box
        area = (x2 - x1) * (y2 - y1)
        idxs = np.argsort(sc)
        
        # remove the boxes of which the score is already too small
        id_to_remove = np.nonzero(sc < obj_threshold)
        idxs = idxs[~np.in1d(idxs,id_to_remove)]

        # keep looping while some indexes still remain in the indexes
        # list
        while len(idxs) > 0:
            # grab the last index in the indexes list and add the
            # index value to the list of picked indexes
            i = idxs[-1]
            pick.append(i)
            sc_output[i] = sc[i]

            # Remove the index picked up from the list of index to explore
            idxs = idxs[:-1]
            
            # find the largest (x, y) coordinates for the start of
            # the bounding box and the smallest (x, y) coordinates
            # for the end of the bounding box
            xx1 = np.maximum(x1[i], x1[idxs])
            yy1 = np.maximum(y1[i], y1[idxs])
            xx2 = np.minimum(x2[i], x2[idxs])
            yy2 = np.minimum(y2[i], y2[idxs])

            # compute the width and height of the bounding box
            w = np.maximum(0, xx2 - xx1)
            h = np.maximum(0, yy2 - yy1)

            # compute the ratio of overlap
            iou = (w * h) / (area[idxs] + area[i] - (w * h))

            # reducce the score of the ones that have overlap
            if method == 'linear':
                sc[idxs] *= 1 - iou
            elif method == 'gaussian':
                sc[idxs] *= np.exp(-(iou * iou)/sigma)
            else: # traditional method
                sc[idxs] *= iou < iou_threshold

            # delete all indexes for which the score is too low
            id_to_remove = np.nonzero(sc < obj_threshold)
            idxs = idxs[~np.in1d(idxs,id_to_remove)]
        
    # return only the bounding boxes that were picked using the
    # integer data type
    return boxes[pick], pick, sc_output

In [173]:
def nms_multiclass(org_boxes, iou_threshold, obj_threshold, sigma=0.5, method=''):
    
    boxes = np.copy(org_boxes)
    
    nb_class = len(boxes[0][5:])
    
    for idx in range(nb_class):
        # Isolate bounding boxes and specific class
        class_array = np.concatenate((boxes[:,:4], boxes[:,[5+idx]]), axis=1)
        
        # Run NMS algorithm for selected class
        _, _, new_cls_scr = nms_fast(class_array, std_iou_threshold, std_obj_threshold, sigma=sigma, method=method)
        
        # Replace score by new scores
        boxes[:, 5+idx] = new_cls_scr
    
    return boxes

In [148]:
def save_pred(dir_path, filename, pred, im_shape, classes):
    
    if not path.isdir(dir_path):
        os.makedirs(dir_path)
    
    csv_path = path.join(dir_path, filename + '.csv')
    
    pred_list = convert_array_to_pred(pred, im_shape, classes)
    
    with open(csv_path, "w") as csv_file:
        writer = csv.writer(csv_file, delimiter=',')
        writer.writerows(pred_list)

In [179]:
# Import list predictions
list_file = [filename[:-len(pred_ext)] for filename in os.listdir(path_pred) if filename.endswith(pred_ext)]

print(len(list_file), ' files to convert.')
print(list_file)

(5, ' files to convert.')
['00025', '00037', '00008', '00089', '00028']


In [187]:
time_std        = np.zeros(len(list_file))
time_soft_lin   = np.zeros(len(list_file))
time_soft_gaus  = np.zeros(len(list_file))

# Convert all the predictions
for idx in trange(len(list_file)):
    filename = list_file[idx]
    filepath = path.join(path_pred, filename + pred_ext)

    image = cv2.imread(path.join(path_image, filename + '.png'))

    # Import all bboxes
    all_boxes = import_pred(filepath)
    
    # --- Standard NMS ---
    start_time_std = time.time()
    nms_boxes = nms_multiclass(all_boxes, std_iou_threshold, std_obj_threshold, method='standard')
    time_std[idx] = time.time() - start_time_std
    # Save predictions to disk
    path_std_nms = path.join(path_nms, 'std_nms')
    save_pred(path_std_nms, filename, nms_boxes, image.shape, classes)
    
    # --- Soft NMS (Linear)---
    start_time_soft_lin = time.time()
    soft_lin_boxes = nms_multiclass(all_boxes, soft_lin_iou_threshold, soft_lin_obj_threshold,method='linear')
    time_soft_lin[idx] = time.time() - start_time_soft_lin
    # Save predictions to disk
    path_soft_lin_nms = path.join(path_nms, 'soft_lin_nms')
    save_pred(path_soft_lin_nms, filename, soft_lin_boxes, image.shape, classes)
    
    # --- Soft NMS (Gaussian)---
    start_time_soft_gaus = time.time()
    soft_gaus_boxes = nms_multiclass(all_boxes, soft_gaus_iou_threshold, soft_gaus_obj_threshold,sigma=soft_gaus_sigma ,method='gaussian')
    time_soft_gaus[idx] = time.time() - start_time_soft_gaus
    # Save predictions to disk
    path_soft_gaus_nms = path.join(path_nms, 'soft_gaus_nms')
    save_pred(path_soft_gaus_nms, filename, soft_gaus_boxes, image.shape, classes)
    
    # Display the results
    if display_result:
        std_pred         = convert_array_to_pred(nms_boxes, image.shape, classes)
        soft_lin_pred    = convert_array_to_pred(soft_lin_boxes, image.shape, classes)
        soft_gaus_pred   = convert_array_to_pred(soft_gaus_boxes, image.shape, classes)

        print('Image: ' + filename)
        print('Standard NMS:')
        print(std_pred)
        print('Soft NMS (Linear):')
        print(soft_lin_pred)
        print('Soft NMS (Gaussian):')
        print(soft_gaus_pred)
        print('-------------------')

# Display average time
print ('Average Time:')
print(' - Standard NMS       : ' + str(np.mean(time_std)))
print(' - Soft NMS (Linear)  : ' + str(np.mean(time_soft_lin)))
print(' - Soft NMS (Gaussian): ' + str(np.mean(time_soft_gaus)))

100%|██████████| 5/5 [00:00<00:00, 15.89it/s]

Average Time:
 - Standard NMS       : 0.00123782157898
 - Soft NMS (Linear)  : 0.00143465995789
 - Soft NMS (Gaussian): 0.00132479667664
